In [1]:
# this one is used as a source of truth for calculating the peformance of the work


In [3]:
import pandas as pd
import numpy as np
import math 
import statistics
from pandas import Timestamp
import math 

In [104]:
# get stat for -old or new and then compare
def get_stat_for(file, authors, label='-new', extra='', time_a = 1, time_b = 1, time_c = 1):
    pd.set_option("display.max_columns", None)
    df = pd.read_csv(file)
 
    count = 0
    total = 0

    for (index, row) in df.iterrows():
        # calcualte total scores
        total_scores = {'history': 0, 'api': 0, 'code': 0, 'api_s': 0}

        for i in range(1, 11):
            h = row['history_at_' + str(i) + '_v']
            c = row['code_at_' + str(i) + '_v']
            a = row['api_at_' + str(i) + '_v']

            if str(h) != 'nan':
                total_scores['history'] += h
            if str(c) != 'nan':
                total_scores['code'] += c
            if str(a) != 'nan':
                total_scores['api'] += a
                
        # calcualte and rank developer scores
        scores = {}        
        for i in range(1, 11):
            h = row['history_at_' + str(i) + '_v']
            h_developer = row['history_at_' + str(i)]
            c = row['code_at_' + str(i) + '_v']
            c_developer = row['code_at_' + str(i)]
            a = row['api_at_' + str(i) + '_v']
            a_developer = row['api_at_' + str(i)]

            if str(h_developer) != 'nan':
                if str(h) == 'nan' or h == 0 or total_scores['history'] <= 0:
                    scores[a_developer] = scores.get(a_developer, []) + [0]
                else:
                    scores[h_developer] = scores.get(h_developer, []) + [time_a * float(h * 100 / total_scores['history'])]

            if str(c_developer) != 'nan':
                if str(c) == 'nan' or c == 0 or total_scores['code'] <= 0:
                    scores[a_developer] = scores.get(a_developer, []) + [0]
                else:
                    scores[c_developer] = scores.get(c_developer, []) + [time_b * float(c * 100 / total_scores['code'])]


            if str(a_developer) != 'nan':
                if str(a) == 'nan' or a == 0 or total_scores['api'] <= 0:
                    scores[a_developer] = scores.get(a_developer, []) + [0]
                else: #  row['jaccard_score']
                    scores[a_developer] = scores.get(a_developer, []) + [time_c * float(a * 100 / total_scores['api'])]
                    pass

                
        # calculate the total
        for index_, score in scores.items():
            if len(score) == 0:
                scores[index_] = 0
            else:
                scores[index_] = sum(score)  ## ** (1/float(len(score)))


        sort = sorted(scores.items(), key=lambda x: x[1], reverse=True)

        chosen = ''
        if len(sort) != 0:
            chosen = sort[0][0]

        if chosen == row['assignees']:
            count += 1

            
        previous = row['assignees']
        total += 1

        if row['assignees'] not in authors:
            authors[row['assignees']] = {
                'total-new': 0, 'count_differently-new': 0, 'count-new': 0, 'count_history-new': 0, 'count_code-new': 0, 'count_api-new': 0,
                'total-old': 0, 'count_differently-old': 0, 'count-old': 0, 'count_history-old': 0, 'count_code-old': 0, 'count_api-old': 0,
                }

        authors[row['assignees']]['total' + label] += 1
        if row['assignees'] == row['history_at_1']:
            authors[row['assignees']]['count_history' + label] += 1
        if row['assignees'] == row['code_at_1']:
            authors[row['assignees']]['count_code' + label] += 1
        if row['assignees'] == row['api_at_1']:
            authors[row['assignees']]['count_api' + label] += 1
        if 1 == row['at_1']:
            authors[row['assignees']]['count' + label] += 1
        if 0 < len(sort) and sort[0][0] == row['assignees']:
            authors[row['assignees']]['count_differently' + label] += 1
            

        
        previous = row['chosen']
        previous_time = row['report_time']
        previous_component = row['component']
   

    if extra == 'ratio':
        return count / total

    print(label)
    print(count)
    print(count / total)
    print(total)
    return authors



authors_ = {}
authors_ = get_stat_for('../../data/output/jdt_direct.csv', authors_, '-new', '', 1, 1, 2)
#authors_ = get_stat_for('../../data/output/jdt_direct-without-util.csv', authors_, '-old')


-new
273
0.455
600


In [92]:
from itertools import product
x=10
y=1
tries = [p/x for p in range(0, int(x*y))] + [1.0]
listOLists = [tries, tries, tries]

best_ratio = 0
best_ratio_code = 0
for list in product(*listOLists):
    ratio = get_stat_for('../../data/output/jdt_direct.csv', {}, '-new', 'ratio', list[0], list[1], list[2])
    print(ratio)
    if best_ratio < ratio:
        best_ratio = ratio
        best_ratio_code = str(list[0]) + '_' + str(list[1]) + '_' + str(list[2])

print('result:')
print(best_ratio)
print(best_ratio_code)

0.23166666666666666
0.36833333333333335
0.36833333333333335
0.36833333333333335
0.36833333333333335
0.36833333333333335
0.36833333333333335
0.36833333333333335
0.36833333333333335
0.36833333333333335
0.36833333333333335
0.42333333333333334
0.425
0.435
0.43833333333333335
0.44666666666666666
0.455
0.4483333333333333
0.45166666666666666
0.4533333333333333
0.4533333333333333
0.45166666666666666
0.42333333333333334
0.42833333333333334
0.425
0.43
0.435
0.43833333333333335
0.43833333333333335
0.445
0.44666666666666666
0.45
0.455
0.42333333333333334
0.4266666666666667
0.42833333333333334
0.425
0.42333333333333334
0.43166666666666664
0.435
0.43833333333333335
0.43833333333333335
0.43833333333333335
0.44
0.42333333333333334
0.4266666666666667
0.42833333333333334
0.425
0.425
0.425
0.43
0.43166666666666664
0.435
0.43833333333333335
0.43833333333333335
0.42333333333333334
0.425
0.42833333333333334
0.42833333333333334
0.425
0.425
0.425
0.4266666666666667
0.43166666666666664
0.43333333333333335
0.43

0.475
0.475
0.47333333333333333
0.47333333333333333
0.48
0.4766666666666667
0.48
0.465
0.4666666666666667
0.4666666666666667
0.465
0.46166666666666667
0.465
0.4633333333333333
0.4716666666666667
0.4716666666666667
0.4716666666666667
0.4716666666666667
0.46
0.4633333333333333
0.46166666666666667
0.465
0.4683333333333333
0.465
0.465
0.4666666666666667
0.4666666666666667
0.4716666666666667
0.47
0.45666666666666667
0.4583333333333333
0.46166666666666667
0.4633333333333333
0.4666666666666667
0.465
0.4666666666666667
0.4683333333333333
0.4683333333333333
0.4666666666666667
0.4666666666666667
0.4583333333333333
0.46
0.46166666666666667
0.4633333333333333
0.465
0.465
0.4633333333333333
0.4666666666666667
0.465
0.4633333333333333
0.4666666666666667
0.4533333333333333
0.45666666666666667
0.4583333333333333
0.4633333333333333
0.4633333333333333
0.465
0.4666666666666667
0.465
0.465
0.46166666666666667
0.4666666666666667
0.46
0.4583333333333333
0.46166666666666667
0.465
0.465
0.465
0.46666666666666

0.4666666666666667
0.4666666666666667
0.465
0.4633333333333333
0.46166666666666667
0.46166666666666667
0.465
0.23166666666666666
0.25
0.27666666666666667
0.2866666666666667
0.2916666666666667
0.3016666666666667
0.31
0.31666666666666665
0.33
0.3416666666666667
0.3466666666666667
0.3283333333333333
0.34
0.3516666666666667
0.35833333333333334
0.36833333333333335
0.37666666666666665
0.38666666666666666
0.3933333333333333
0.39666666666666667
0.40166666666666667
0.4033333333333333
0.4033333333333333
0.405
0.41
0.4216666666666667
0.42833333333333334
0.43166666666666664
0.43333333333333335
0.435
0.43
0.435
0.43833333333333335
0.43333333333333335
0.43833333333333335
0.43666666666666665
0.44333333333333336
0.4483333333333333
0.4483333333333333
0.45
0.45
0.4483333333333333
0.4483333333333333
0.4483333333333333
0.45
0.4533333333333333
0.4533333333333333
0.45666666666666667
0.455
0.4533333333333333
0.4533333333333333
0.4533333333333333
0.4533333333333333
0.455
0.4583333333333333
0.4533333333333333


In [105]:
from itertools import product
x=10
y=1
tries = [p/x for p in range(0, int(x*y))] + [1.0]
listOLists = [tries, tries, tries]

best_ratio = 0
best_ratio_code = 0
for list in product(*listOLists):
    ratio = get_stat_for('../../data/output/jdt_direct.csv', {}, '-new', 'ratio', list[0], list[1], list[2])
    print(ratio)
    if best_ratio < ratio:
        best_ratio = ratio
        best_ratio_code = str(list[0]) + '_' + str(list[1]) + '_' + str(list[2])

print('result:')
print(best_ratio)
print(best_ratio_code)

0.23166666666666666
0.36833333333333335
0.36833333333333335
0.36833333333333335
0.36833333333333335
0.36833333333333335
0.36833333333333335
0.36833333333333335
0.36833333333333335
0.36833333333333335
0.36833333333333335
0.42333333333333334
0.43833333333333335
0.435
0.43666666666666665
0.4266666666666667
0.42333333333333334
0.42333333333333334
0.4216666666666667
0.4216666666666667
0.42
0.4166666666666667
0.42333333333333334
0.43833333333333335
0.43833333333333335
0.435
0.435
0.435
0.43666666666666665
0.42833333333333334
0.4266666666666667
0.42333333333333334
0.42333333333333334
0.42333333333333334
0.44
0.43833333333333335
0.43833333333333335
0.43833333333333335
0.43333333333333335
0.435
0.435
0.435
0.43666666666666665
0.43166666666666664
0.42333333333333334
0.43666666666666665
0.43833333333333335
0.44
0.43833333333333335
0.44333333333333336
0.435
0.43333333333333335
0.435
0.43666666666666665
0.435
0.42333333333333334
0.43166666666666664
0.43833333333333335
0.435
0.44166666666666665
0.43

0.4666666666666667
0.4683333333333333
0.4666666666666667
0.46
0.45666666666666667
0.4583333333333333
0.4583333333333333
0.455
0.45166666666666666
0.4483333333333333
0.465
0.46166666666666667
0.46166666666666667
0.4683333333333333
0.45666666666666667
0.4533333333333333
0.45666666666666667
0.455
0.45166666666666666
0.4483333333333333
0.4483333333333333
0.46
0.465
0.465
0.46166666666666667
0.4633333333333333
0.46
0.4583333333333333
0.455
0.45166666666666666
0.4483333333333333
0.4483333333333333
0.45666666666666667
0.4683333333333333
0.4666666666666667
0.46
0.46
0.455
0.45666666666666667
0.46
0.45666666666666667
0.45166666666666666
0.4483333333333333
0.4583333333333333
0.465
0.465
0.46
0.455
0.455
0.4583333333333333
0.45666666666666667
0.45166666666666666
0.4533333333333333
0.45166666666666666
0.4533333333333333
0.4633333333333333
0.46
0.46
0.4583333333333333
0.45166666666666666
0.4533333333333333
0.455
0.455
0.45166666666666666
0.44666666666666666
0.46
0.465
0.4633333333333333
0.461666666

0.46166666666666667
0.45666666666666667
0.455
0.4533333333333333
0.23166666666666666
0.31333333333333335
0.36833333333333335
0.3933333333333333
0.4
0.405
0.4116666666666667
0.4083333333333333
0.405
0.405
0.4066666666666667
0.3283333333333333
0.395
0.4116666666666667
0.42
0.425
0.43
0.43166666666666664
0.43166666666666664
0.42333333333333334
0.42333333333333334
0.42
0.4033333333333333
0.43333333333333335
0.445
0.4483333333333333
0.45
0.45
0.45166666666666666
0.44666666666666666
0.44333333333333336
0.44166666666666665
0.43333333333333335
0.43333333333333335
0.4533333333333333
0.45
0.455
0.46
0.4666666666666667
0.46
0.45666666666666667
0.45
0.4533333333333333
0.4483333333333333
0.45
0.455
0.4583333333333333
0.4633333333333333
0.47333333333333333
0.4683333333333333
0.4633333333333333
0.46166666666666667
0.4583333333333333
0.46
0.4583333333333333
0.4533333333333333
0.455
0.46166666666666667
0.4633333333333333
0.47
0.4666666666666667
0.465
0.4666666666666667
0.46
0.4633333333333333
0.4566666